In [ ]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import datetime as dt
import pandas as pd
import random
from datetime import timezone,datetime

In [ ]:
#Taken from tutorial
def publish_message(producer_instance, topic_name, value):
    try:
        producer_instance.send(topic_name, value=value)
#         producer_instance.flush()
#         print('Message published successfully. Data: ' + str(value))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [ ]:
#Function to send an attack per machine using dictionary to keep track of indexes
def send_attack_per_machine(df_machine,start_index_dict,end_index_dict):
    #Which machine to send rows from
    key = df_machine.machine[0]
    #Should number of rows sent for each rows be random or does the randomness comes a step earlier?
    rowsToSend = random.randrange(100,500)
    #Size of the given df
    totalRows = int(df_machine.shape[0])
#     print('Machine No: %d'%key)
#     print('Rows to send: %d'%(rowsToSend))
    #Add to the end index of the selected machine 
    end_index_dict[key] = end_index_dict[key]+rowsToSend
    #Get the start index of the machine from previous run
    start_index = start_index_dict[key]
    #Get end index from previous run
    end_index = end_index_dict[key]
    #Check whether end index is lesser than total number of rows in df
    if(end_index>totalRows):
#         print('second')
        #Set start_index of the particular machine to 0 in the dict
        start_index_dict[key] = 0
        #Get the start_index which is now 0
        start_index = start_index_dict[key]
        #update the end_index for the particular machine to rowsToSend in the dictt
        end_index_dict[key] = rowsToSend
        #get the end index
        end_index = end_index_dict[key]
#         print('end index: %d'%(end_index))
#         print('start index: %d'%(start_index))
        #Create a temp df with the selected rows to be sent
    temp = df_machine.iloc[start_index:end_index,:]
    #appened the ts in unix-timestamp
    temp['ts'] = int(datetime.now().replace(tzinfo=timezone.utc).timestamp())
    #publish the message to kafka stream
    publish_message(producer, topic, temp.to_dict(orient='record'))
    #update the start_index to end_index
    start_index_dict[key] = end_index
#     print('start_index_dict :%s'%(str(start_index_dict)))
#     print('end_index_dict :%s'%(str(end_index_dict)))

In [ ]:
start_index_dict = {4:0, 5:0, 6:0, 7:0, 8:0}
end_index_dict = {4:0, 5:0, 6:1, 7:0, 8:0}
producer = connect_kafka_producer()
topic = 'Assignment2B-process'
df_process = pd.read_csv('Streaming_Linux_process.csv')
df_machine_4 = df_process.loc[df_process['machine']==4].reset_index()
df_machine_5 = df_process.loc[df_process['machine']==5].reset_index()
df_machine_6 = df_process.loc[df_process['machine']==6].reset_index()
df_machine_7 = df_process.loc[df_process['machine']==7].reset_index()
df_machine_8 = df_process.loc[df_process['machine']==8].reset_index()

In [ ]:
df_process.groupby('machine').count()

In [ ]:
while True:
    send_attack_per_machine(df_machine_4,start_index_dict,end_index_dict)
    send_attack_per_machine(df_machine_5,start_index_dict,end_index_dict)
    send_attack_per_machine(df_machine_6,start_index_dict,end_index_dict)
    send_attack_per_machine(df_machine_7,start_index_dict,end_index_dict)
    send_attack_per_machine(df_machine_8,start_index_dict,end_index_dict)
    sleep(5)